In [2]:
import pandas as pd;
import numpy as np;
import sklearn as sk;
import xgboost as xgb;
import matplotlib as plt;
import seaborn as sns;
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [3]:
#Cargamos el dataset desde el archivo csv
df = pd.read_csv(r'C:\Users\mrodr\OneDrive\Escritorio\UAL\TFG\prestamos campos definidos\loan.csv')

C:\Users\mrodr\AppData\Local\Temp\ipykernel_14904\682258873.py:2: DtypeWarning: Columns (19,47,55,112,123,124,125,128,129,130,133,139,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\mrodr\OneDrive\Escritorio\UAL\TFG\prestamos campos definidos\loan.csv')


In [11]:
#Veamos la cabecera del dataset
pd.set_option('display.max_columns', None)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0.0,Apr-2001,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,NaN,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0.0,Jun-1987,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt cons

In [19]:
# Verificar filas duplicadas
duplicados = df.duplicated()
print(f"Número de filas duplicadas: {duplicados.sum()}")

Número de filas duplicadas: 0


In [12]:
#Veamos los valores únicos de loan_status
print(df['loan_status'].unique())
#Cuanta cantidad de valores únicos hay
print(df["loan_status"].value_counts())

['Current' 'Fully Paid' 'Late (31-120 days)' 'In Grace Period'
 'Charged Off' 'Late (16-30 days)' 'Default'
 'Does not meet the credit policy. Status:Fully Paid'
 'Does not meet the credit policy. Status:Charged Off']
loan_status
Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: count, dtype: int64


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Columns: 145 entries, id to settlement_term
dtypes: float64(105), int64(4), object(36)
memory usage: 2.4+ GB


In [14]:
#LIMPIEZA DE DATOS
#Vamos a eliminar filas vacias
#df = df.dropna()

# Definir las categorías de Good Loan y Bad Loan
good_loan_status = ["Current", "Fully Paid", "Does not meet the credit policy. Status:Fully Paid"]
bad_loan_status = ["Late (31-120 days)", "In Grace Period", "Charged Off", 
                   "Late (16-30 days)", "Default", 
                   "Does not meet the credit policy. Status:Charged Off"]

# Transformar 'loan_status' a binario
df['loan_status'] = df['loan_status'].apply(lambda x: 1 if x in good_loan_status else 0)

# Verificar la transformación
print(df['loan_status'].value_counts())
df.head()

loan_status
1    1963635
0     297033
Name: count, dtype: int64


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,1,n,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0.0,Apr-2001,1.0,NaN,45.0,9.0,1.0,4341,10.3,34.0,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,0.0,16901.0,2.0,2.0,1.0,2.0,2.0,12560.0,69.0,2.0,7.0,2137.0,28.0,42000.0,1.0,11.0,2.0,9.0,1878.0,34360.0,5.9,0.0,0.0,140.0,212.0,1.0,1.0,0.0,1.0,NaN,2.0,NaN,0.0,2.0,5.0,3.0,3.0,16.0,7.0,18.0,5.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,60124.0,16901.0,36500.0,18124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,1,n,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0.0,Jun-1987,0.0,71.0,75.0,13.0,1.0,12315,24.2,44.0,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0.0,NaN,1,Individual,NaN,NaN,NaN,0.0,1208.0,321915.0,4.0,4.0,2.0,3.0,3.0,87153.0,88.0,4.0,5.0,998.0,57.0,50800.0,2.0,15.0,2.0,10.0,24763.0,13761.0,8.3,0.0,0.0,163.0,378.0,4.0,3.0,3.0,4.0,NaN,4.0,NaN,0.0,2.0,4.0,4.0,9.0,27.0,8.0,14.0,4.0,13.0,0.0,0.0,0.0,6.0,95.0,0.0,1.0,0.0,372872.0,99468.0,15000.0,94072.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,1,n,NaN,NaN,debt_consolidation,Debt consolidation,490xx,MI

In [15]:
# Verificar los tipos de datos de cada columna
print(df.dtypes.value_counts())

#RESULTADOS
#Vemos que hay 3 tipos de datos: float64, int64 y object
#Vamos a ver cuantos valores únicos hay en cada columna de tipo object

#NUMERICAS  110
#FLOAT      105
#INT        5

#OBJECT     35

float64    105
object      35
int64        5
Name: count, dtype: int64


In [16]:
#Una vez ya sabemos los tipo de datos que tenmos vamos a ver la correalción de las variables con loan_status
#Separamos las variables numéricas y las categóricas
variableNumerica = df.select_dtypes(include=['int64', 'float64']).columns
variableCategorica = df.select_dtypes(include=['object']).columns

In [17]:
#Calculamos la correlación de las variables numéricas con loan_status
correlation_matrix = df[variableNumerica].corr()

# Extraer la correlación de cada variable con 'loan_status'
correlation_with_target = correlation_matrix['loan_status'].sort_values(ascending=False)

In [18]:
#Mostramos las correlaciones
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(correlation_with_target.sort_values(ascending=False))

loan_status                                   1.000000
total_rec_prncp                               0.223022
last_pymnt_amnt                               0.187750
out_prncp                                     0.162198
out_prncp_inv                                 0.162182
total_pymnt_inv                               0.137199
total_pymnt                                   0.137170
bc_open_to_buy                                0.082975
total_bc_limit                                0.075750
sec_app_mths_since_last_major_derog           0.072526
tot_hi_cred_lim                               0.062747
mths_since_recent_inq                         0.061419
total_rev_hi_lim                              0.057065
avg_cur_bal                                   0.057003
mo_sin_rcnt_tl                                0.055322
mo_sin_rcnt_rev_tl_op                         0.054432
tot_cur_bal                                   0.052712
sec_app_mort_acc                              0.051780
settlement

In [11]:
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd

# Función para calcular Cramér's V
def cramers_v(x, y):
    """
    Calcula la correlación de Cramér's V entre dos variables categóricas.

    Parámetros:
    x (pd.Series): Primera variable categórica.
    y (pd.Series): Segunda variable categórica.

    Retorna:
    float: Valor de Cramér's V.
    """
    # Crear la matriz de contingencia
    confusion_matrix = pd.crosstab(x, y)
    
    # Calcular el estadístico chi-cuadrado
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()  # Total de observaciones
    
    # Calcular phi2
    phi2 = chi2 / n
    
    # Ajustar phi2 para evitar valores negativos
    r, k = confusion_matrix.shape  # Número de filas y columnas
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    
    # Ajustar el número de filas y columnas
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    
    # Calcular el denominador
    denominador = min((kcorr-1), (rcorr-1))
    
    # Manejar divisiones por cero y valores NaN
    if denominador == 0 or np.isnan(denominador):
        return np.nan  # Devolver NaN si el denominador es 0 o NaN
    
    # Calcular Cramér's V
    return np.sqrt(phi2corr / denominador)

# Calcular Cramér's V para cada variable categórica
cramer_results = {}
for var in variableCategorica:
    cramer_results[var] = cramers_v(df[var], df['loan_status'])

# Mostrar resultados
print("Resultados de Cramér's V:")
for var, v in cramer_results.items():
    print(f"{var}: {v:.4f}")


Resultados de Cramér's V:
term: 0.0911
grade: 0.2302
sub_grade: 0.2346
emp_title: 0.1817
emp_length: 0.0170
home_ownership: 0.0524
verification_status: 0.0956
issue_d: 0.1731
pymnt_plan: 0.0443
desc: 0.0000
purpose: 0.0510
title: 0.0869
zip_code: 0.0498
addr_state: 0.0384
earliest_cr_line: 0.0347
initial_list_status: 0.0606
last_pymnt_d: 0.3097
next_pymnt_d: 0.0918
last_credit_pull_d: 0.3982
application_type: 0.0375
verification_status_joint: 0.0058
sec_app_earliest_cr_line: 0.0321
hardship_flag: 0.0502
hardship_type: nan
hardship_reason: 0.3272
hardship_status: 0.4789
hardship_start_date: 0.3514
hardship_end_date: 0.3464
payment_plan_start_date: 0.3472
hardship_loan_status: 0.2770
disbursement_method: 0.0585
debt_settlement_flag: 0.3129
debt_settlement_flag_date: 0.0000
settlement_status: 0.0281
settlement_date: 0.0000


In [13]:
# Mostrar las correlaciones ordenadas
print(pd.Series(cramer_results).sort_values(ascending=False))

hardship_status              0.478854
last_credit_pull_d           0.398188
hardship_start_date          0.351415
payment_plan_start_date      0.347165
hardship_end_date            0.346425
hardship_reason              0.327185
debt_settlement_flag         0.312886
last_pymnt_d                 0.309678
hardship_loan_status         0.276995
sub_grade                    0.234587
grade                        0.230237
emp_title                    0.181675
issue_d                      0.173060
verification_status          0.095602
next_pymnt_d                 0.091796
term                         0.091056
title                        0.086877
initial_list_status          0.060551
disbursement_method          0.058471
home_ownership               0.052392
purpose                      0.050961
hardship_flag                0.050183
zip_code                     0.049814
pymnt_plan                   0.044321
addr_state                   0.038434
application_type             0.037494
earliest_cr_

In [14]:
#CLASIFICAMOS LAS VARIABLES

# Definir umbrales
umbral_fuerte = 0.2
umbral_moderado = 0.1

# Inicializar listas para cada categoría NUMERICAS
variables_fuertes_numericas = []
variables_moderadas_numericas = []
variables_sin_relacion_numericas = []

# Clasificar variables numéricas
for variable, correlacion in correlation_with_target.items():
    if pd.isna(correlacion):  # Si es NaN
        variables_sin_relacion_numericas.append(variable)
    elif abs(correlacion) > umbral_fuerte:  # Correlación fuerte
        variables_fuertes_numericas.append(variable)
    elif umbral_moderado < abs(correlacion) <= umbral_fuerte:  # Correlación moderada
        variables_moderadas_numericas.append(variable)
    else:  # Correlación débil o sin relación
        variables_sin_relacion_numericas.append(variable)

# Inicializar listas para cada categoría CATEGORICAS
variables_fuertes_categoricas = []
variables_moderadas_categoricas = []
variables_sin_relacion_categoricas = []

# Clasificar variables categóricas
for variable, cramer_v in cramer_results.items():
    if pd.isna(cramer_v):  # Si es NaN
        variables_sin_relacion_categoricas.append(variable)
    elif cramer_v > umbral_fuerte:  # Correlación fuerte
        variables_fuertes_categoricas.append(variable)
    elif umbral_moderado < cramer_v <= umbral_fuerte:  # Correlación moderada
        variables_moderadas_categoricas.append(variable)
    else:  # Correlación débil o sin relación
        variables_sin_relacion_categoricas.append(variable)

In [15]:
#df[variables_fuertes_categoricas] = df[variables_fuertes_categoricas].astype('category')

# Unir las variables fuertes numéricas y categóricas
variables_fuertes_totales = variables_fuertes_numericas + variables_fuertes_categoricas

# Unir las variables moderadas numéricas y categóricas
variables_moderadas_totales = variables_moderadas_numericas + variables_moderadas_categoricas

# Unir las variables sin relación numéricas y categóricas
variables_sin_relacion_totales = variables_sin_relacion_numericas + variables_sin_relacion_categoricas


#Fuertes Categoricas
print("Variables fuertes categóricas:", variables_fuertes_categoricas, len(variables_fuertes_categoricas))
#Fuertes Numericas
print("Variables fuertes numéricas:", variables_fuertes_numericas, len(variables_fuertes_numericas))

# Mostrar los resultados
print("Variables fuertes totales:", variables_fuertes_totales)
print("Variables moderadas totales:", variables_moderadas_totales)
print("Variables sin relación totales:", variables_sin_relacion_totales)

Variables fuertes categóricas: ['grade', 'sub_grade', 'last_pymnt_d', 'last_credit_pull_d', 'hardship_reason', 'hardship_status', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status', 'debt_settlement_flag'] 11
Variables fuertes numéricas: ['loan_status', 'total_rec_prncp', 'int_rate', 'hardship_dpd', 'collection_recovery_fee', 'recoveries'] 6
Variables fuertes totales: ['loan_status', 'total_rec_prncp', 'int_rate', 'hardship_dpd', 'collection_recovery_fee', 'recoveries', 'grade', 'sub_grade', 'last_pymnt_d', 'last_credit_pull_d', 'hardship_reason', 'hardship_status', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status', 'debt_settlement_flag']
Variables moderadas totales: ['last_pymnt_amnt', 'out_prncp', 'out_prncp_inv', 'total_pymnt_inv', 'total_pymnt', 'total_rec_late_fee', 'emp_title', 'issue_d']
Variables sin relación totales: ['bc_open_to_buy', 'total_bc_limit', 'sec_app_mths_since_last_major_derog'

In [16]:
# Contar el número de variables en cada categoría
num_variables_fuertes = len(variables_fuertes_totales)
num_variables_moderadas = len(variables_moderadas_totales)
num_variables_sin_relacion = len(variables_sin_relacion_totales)

# Mostrar los resultados
print(f"Número de variables con correlación fuerte: {num_variables_fuertes}")
print(f"Número de variables con correlación moderada: {num_variables_moderadas}")
print(f"Número de variables sin relación: {num_variables_sin_relacion}")

print("\nTotal de variables:", num_variables_fuertes + num_variables_moderadas + num_variables_sin_relacion)
print("Total de variables utiles:", num_variables_fuertes + num_variables_moderadas)

Número de variables con correlación fuerte: 17
Número de variables con correlación moderada: 8
Número de variables sin relación: 120

Total de variables: 145
Total de variables utiles: 25


In [17]:
# Mostrar los resultados
print("Variables con correlación fuerte (|correlación| > 0.2 o Cramér's V > 0.2):")
print(variables_fuertes_totales)

print("\nVariables con correlación moderada (0.1 < |correlación| <= 0.2 o 0.1 < Cramér's V <= 0.2):")
print(variables_moderadas_totales)

print("\nVariables sin relación (|correlación| <= 0.1, Cramér's V <= 0.1 o NaN):")
print(variables_sin_relacion_totales)

Variables con correlación fuerte (|correlación| > 0.2 o Cramér's V > 0.2):
['loan_status', 'total_rec_prncp', 'int_rate', 'hardship_dpd', 'collection_recovery_fee', 'recoveries', 'grade', 'sub_grade', 'last_pymnt_d', 'last_credit_pull_d', 'hardship_reason', 'hardship_status', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status', 'debt_settlement_flag']

Variables con correlación moderada (0.1 < |correlación| <= 0.2 o 0.1 < Cramér's V <= 0.2):
['last_pymnt_amnt', 'out_prncp', 'out_prncp_inv', 'total_pymnt_inv', 'total_pymnt', 'total_rec_late_fee', 'emp_title', 'issue_d']

Variables sin relación (|correlación| <= 0.1, Cramér's V <= 0.1 o NaN):
['bc_open_to_buy', 'total_bc_limit', 'sec_app_mths_since_last_major_derog', 'tot_hi_cred_lim', 'mths_since_recent_inq', 'total_rev_hi_lim', 'avg_cur_bal', 'mo_sin_rcnt_tl', 'mo_sin_rcnt_rev_tl_op', 'tot_cur_bal', 'sec_app_mort_acc', 'settlement_percentage', 'annual_inc_joint', 'mths_since_recent_bc', 'mort_a

In [18]:
#Manejar multicolinealidad

# Calcular la matriz de correlación entre las variables fuertes
correlation_matrix_fuertes = df[variables_fuertes_numericas].corr()

# Definir un umbral para la correlación alta
umbral_correlacion = 0.7

# Identificar pares de variables con correlación alta
variables_a_descartar = set()  # Conjunto para almacenar variables a descartar
for i in range(len(correlation_matrix_fuertes.columns)):
    for j in range(i):
        if abs(correlation_matrix_fuertes.iloc[i, j]) > umbral_correlacion:
            colname_i = correlation_matrix_fuertes.columns[i]
            colname_j = correlation_matrix_fuertes.columns[j]
            
            # Comparar la correlación con 'loan_status' y descartar la que tenga menor correlación
            if abs(correlation_with_target[colname_i]) > abs(correlation_with_target[colname_j]):
                variables_a_descartar.add(colname_j)
            else:
                variables_a_descartar.add(colname_i)

# Mostrar las variables a descartar
print("Variables a descartar debido a multicolinealidad:")
print(variables_a_descartar)

# Filtrar las variables fuertes para eliminar las redundantes
variables_fuertes_finales = [var for var in variables_fuertes_numericas if var not in variables_a_descartar]

# Mostrar las variables fuertes finales
print("\nVariables fuertes finales después de eliminar multicolinealidad:")
print(variables_fuertes_finales)

Variables a descartar debido a multicolinealidad:
{'collection_recovery_fee'}

Variables fuertes finales después de eliminar multicolinealidad:
['loan_status', 'total_rec_prncp', 'int_rate', 'hardship_dpd', 'recoveries']


In [19]:
#Seleccionar las variables finales
variables_fuertes_finales_totales = variables_fuertes_finales + variables_fuertes_categoricas
print(variables_fuertes_finales_totales)
lineaTotal= len(variables_fuertes_finales_totales)
print(lineaTotal)

['loan_status', 'total_rec_prncp', 'int_rate', 'hardship_dpd', 'recoveries', 'grade', 'sub_grade', 'last_pymnt_d', 'last_credit_pull_d', 'hardship_reason', 'hardship_status', 'hardship_start_date', 'hardship_end_date', 'payment_plan_start_date', 'hardship_loan_status', 'debt_settlement_flag']
16


In [20]:
#UTILIZAR ENABLE
# Identificar las columnas categóricas (tipo 'object')
columnas_categoricas = df.select_dtypes(include=['object']).columns.tolist()

# Convertir las columnas categóricas a tipo 'category'
df[columnas_categoricas] = df[columnas_categoricas].astype('category')

In [21]:
#COMPROBAMOS SI SE HA CAMBIADO EL TIPO DE DATOS CATEGORICO
print(df.dtypes.value_counts())


float64     105
int64         5
category      2
category      2
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
category      1
Name: count, dtype: int64


In [22]:

# Definir las variables predictoras (X) y la variable objetivo (y)
X = df[variables_fuertes_finales_totales]  # Variables finales seleccionadas
y = df['loan_status']              # Variable objetivo

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar las dimensiones de los conjuntos
print("Conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Conjunto de prueba:", X_test.shape, y_test.shape)

Conjunto de entrenamiento: (1808534, 16) (1808534,)
Conjunto de prueba: (452134, 16) (452134,)


In [23]:
X_train.head()
#X_train.describe()


,loan_status,total_rec_prncp,int_rate,hardship_dpd,recoveries,grade,sub_grade,last_pymnt_d,last_credit_pull_d,hardship_reason,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,debt_settlement_flag
305053,1,2814.29,21.85,NaN,0.0,D,D5,Feb-2019,Feb-2019,NaN,NaN,NaN,NaN,NaN,NaN,N
50247,1,356.23,13.56,NaN,0.0,C,C1,Feb-2019,Feb-2019,NaN,NaN,NaN,NaN,NaN,NaN,N
1630001,1,20000.00,12.74,NaN,0.0,C,C1,Aug-2017,Feb-2019,NaN,NaN,NaN,NaN,NaN,NaN,N
927626,1,8000.00,13.49,NaN,0.0,C,C2,Oct-2017,Jun-2018,NaN,NaN,NaN,NaN,NaN,NaN,N
758488,1,30000.00,11.99,NaN,0.0,C,C1,Feb-2019,Feb-2019,NaN,NaN,NaN,NaN,NaN,NaN,N


In [ ]:
'''
## Identificar las columnas categóricas en X_train
columnas_categoricas = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Aplicar one-hot encoding a las columnas categóricas
encoder = OneHotEncoder(
    drop='first',  # Para evitar multicolinealidad
    sparse_output=False,  # Para obtener un array denso
    handle_unknown='ignore'  # Ignorar categorías desconocidas
)

# Ajustar el encoder en el conjunto de entrenamiento y transformar ambos conjuntos
X_train_encoded = encoder.fit_transform(X_train[columnas_categoricas])
X_test_encoded = encoder.transform(X_test[columnas_categoricas])

# Convertir el resultado a un DataFrame
X_train_encoded = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(columnas_categoricas))
X_test_encoded = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(columnas_categoricas))

# Eliminar las columnas categóricas originales y concatenar las codificadas
X_train = X_train.drop(columnas_categoricas, axis=1).reset_index(drop=True)
X_test = X_test.drop(columnas_categoricas, axis=1).reset_index(drop=True)

X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

# Verificar el resultado
print("Conjunto de entrenamiento después de one-hot encoding:", X_train.shape)
print("Conjunto de prueba después de one-hot encoding:", X_test.shape)'
'''

c:\Users\mrodr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Conjunto de entrenamiento después de one-hot encoding: (1808534, 416)
Conjunto de prueba después de one-hot encoding: (452134, 416)


In [24]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Crear y entrenar el modelo XGBoost
model = XGBClassifier(
    random_state=42,  # Semilla para reproducibilidad
    n_estimators=100,  # Número de árboles
    learning_rate=0.1,  # Tasa de aprendizaje
    max_depth=5,  # Profundidad máxima de los árboles
    subsample=0.8,  # Submuestra del conjunto de datos para cada árbol
    tree_method='hist',  # Método eficiente para datasets grandes
    colsample_bytree=0.8,  # Submuestra de características para cada árbol
    eval_metric='logloss',  # Métrica de evaluación
    enable_categorical=True,  # Habilitar el manejo de variables categóricas
    n_jobs=-1  # Usar todos los núcleos de la CPU
)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para la clase positiva

# Evaluar el modelo
print("Precisión del modelo (XGBoost):", accuracy_score(y_test, y_pred))
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))
print("\nMatriz de confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nAUC-ROC:", roc_auc_score(y_test, y_pred_proba))

Precisión del modelo (XGBoost): 1.0

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     59629
           1       1.00      1.00      1.00    392505

    accuracy                           1.00    452134
   macro avg       1.00      1.00      1.00    452134
weighted avg       1.00      1.00      1.00    452134


Matriz de confusión:
[[ 59629      0]
 [     0 392505]]

AUC-ROC: 1.0


In [25]:
# Verificar la correlación entre las variables predictoras y la variable objetivo
correlation_with_target = X_train.corrwith(y_train)
print("Correlación con la variable objetivo:")
print(correlation_with_target.sort_values(ascending=False))

ValueError: could not convert string to float: 'D'

In [ ]:
#Descargar
X_test.to_csv('X_test.csv', index=False)

In [ ]:
X_train